In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.exceptions import SpotifyException
import time
from random import randint
import json

In [ ]:
def get_track_by_index(pl, idx, str_return = True):
    """
    Returns track ID, track name, and artists for a given playlist and index of track within that playlist.
    """
    track_dict = pl['tracks']['items'][idx]['track']
    track_name = track_dict['name']
    artist_list = track_dict['artists']
    track_id = track_dict['id']
    artists = [x['name'] for x in artist_list]
    if str_return:
        artists = ", ".join(artists)

    return {"id": track_id,
            "name": track_name,
            "artist": artists}


def retrieve_devices(sp_obj):
    """
    Returns available device names and IDs known to Spotify.
    """
    available_devices = sp_obj.devices()
    device_list = available_devices['devices']
    device_id_dict = {x['name']: x['id'] for x in device_list}

    return device_id_dict


def play_track(device_id, track_uri, sp_obj):
    """
    Play specific track_uri on device_id device.
    """
    track_uri_fmt = "spotify:track:%s" % track_uri
    return sp_obj.start_playback(device_id = device_id,
                                 uris = [track_uri_fmt])


def pause_track(device_id, sp_obj):
    """
    Pause specific device_id device.
    """
    return sp_obj.pause_playback(device_id = device_id)


def snippet_playback(snippet_length: int = 15, **kwargs):
    """
    Play track_id for length of snippet_length on given device_id.
    """
    try:
        play_track(**kwargs)
        time.sleep(snippet_length)
        pause_track(device_id = kwargs['device_id'],
                    sp_obj = kwargs['sp_obj'])
    except SpotifyException:
        print("Playback could not be paused. This is either due to playback being paused manually or device disconnection")
    
    return kwargs


def play_random_snippet_pl(pl_data, sp_obj, snippet_length, to_play_device):
    num_tracks = len(pl_data['tracks']['items'])
    track_idx = randint(0, num_tracks - 1)
    track_data = get_track_by_index(pl_data, track_idx)
    track_id = track_data['id']
    available_devices = retrieve_devices(sp_obj)
    to_play_device = 'RatBook Pro'
    to_play_device_id = available_devices[to_play_device]
    snippet_playback(snippet_length = snippet_length,
                     device_id = to_play_device_id,
                     track_uri = track_id,
                     sp_obj = sp_obj)

    return track_data


def playlist_id_retrieve(name, file_name = "playlists.json"):
    """
    Retrieve playlist dict from file_name.
    """
    f = open(file_name)
    playlist_dict = json.load(f)
    if name not in playlist_dict:
        ValueError("Name not valid - please input one of %s" % playlist_dict.keys())
    return playlist_dict[name]

In [ ]:
spotify_client_id = "foo"
spotify_client_secret = "foo"
scope = "user-read-playback-state,user-modify-playback-state"
redirect_uri = "http://localhost:8888/callback"
spotify_oauth = SpotifyOAuth(client_id = spotify_client_id,
                             client_secret = spotify_client_secret,
                             redirect_uri = redirect_uri,
                             scope = scope)
spotify = spotipy.Spotify(client_credentials_manager = spotify_oauth)

In [ ]:
pl_link = playlist_id_retrieve('70s')
pl_data = spotify.playlist(pl_link)

play_random_snippet_pl(pl_data, sp_obj = spotify, snippet_length = 20, to_play_device = 'RatBook Pro')